In [13]:
import numpy as np

In [22]:
from BFS import BFS, BFSmulti

In [15]:
import tqdm
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import os
from mpl_toolkits import mplot3d
from mpl_toolkits.mplot3d import Axes3D
bond_types = ['1JHC', '1JHN', '2JHC', '2JHH', '2JHN', '3JHC', '3JHH', '3JHN']
pickles = [i for i in os.listdir('data') if ".pkl" in i]
#dfs = {i[:-4] : pd.read_pickle('data/'+i) for i in pickles}
#print ("\t".join(dfs.keys()))
print (pickles)

['traintestadj.pkl', 'distances.pkl', 'magnetic_shielding_tensors.pkl', 'traintestpathfixed.pkl', 'df_graph_fixed.pkl', 'test.pkl', 'traintest.pkl', 'dipole_moments.pkl', 'structures_reindexed.pkl', 'lengthes_4.pkl', 'train_contrib.pkl', 'df_graph.pkl', 'train.pkl', 'potential_energy.pkl', 'struc_and_dist.pkl', 'structures.pkl', 'traintestpath.pkl', 'dist_df.pkl', 'mulliken_charges.pkl', 'traintestadjfixed.pkl']


In [16]:
traintest = pd.read_pickle('data/traintest.pkl')

df_graph = pd.read_pickle('data/df_graph_fixed.pkl')

In [16]:
#traintest = pd.concat((train, test), sort=False)

In [19]:
#traintest.to_pickle('data/traintest.pkl')

In [17]:
AdjMatrix = df_graph.AdjMatrix.loc[traintest.molecule_name]

In [18]:
traintest['AdjMatrix'] = AdjMatrix.values

In [20]:
traintest.to_pickle('data/traintestadjfixed.pkl')

In [21]:
%%time
traintestpath=traintest.apply(lambda x: BFS(x.AdjMatrix, x.atom_index_0, x.atom_index_1), axis=1)
traintestpath.to_pickle('data/traintestpathfixed.pkl')

CPU times: user 23min 35s, sys: 2min 9s, total: 25min 44s
Wall time: 22min 56s


In [ ]:
uniq = np.unique(traintestpath.values)

In [2]:
traintestpath = dfs['traintestpath']

In [6]:
np.max( [len(x) for x in traintestpath])

4

In [5]:
np.min( [len(x) for x in traintestpath])

2

In [26]:
from importlib import reload 

In [29]:
import BFS

In [28]:
reload(BFSmulti)

TypeError: reload() argument must be a module

In [31]:
import numpy as np
import queue

def BFSmulti(A, start_idx, end_idx):  #A = adjacency matrix
    q = queue.deque()
    q.append(start_idx)
    predecessors = [ [] for _ in range(A.shape[0]) ]
    distfromstart = np.zeros(A.shape[0], dtype=np.int32) + 99999
    predecessors[start_idx].append(start_idx)
    distfromstart[start_idx] = 0
    while (len(q)):
        current_idx = q.popleft()
        for next_idx in np.argwhere(A[current_idx]):  #returns list of lists of single value
            if len(predecessors[next_idx[0]]) == 0:
                q.append(next_idx[0])
                predecessors[next_idx[0]].append(current_idx)
                distfromstart[next_idx[0]] = distfromstart[current_idx] + 1
            elif (distfromstart[next_idx[0]] == distfromstart[current_idx] + 1):
                predecessors[next_idx[0]].append(current_idx)
                
    if len(predecessors[end_idx]) == 0:
        return None
    
    def give_pathes(ending, ind):
        if ind == start_idx:
            return [ending + [ind]]
        ret_list = []
        for pred_ind in predecessors[ind]:
            ret_list += give_pathes(ending + [ind], pred_ind)
        return ret_list
    
    return [list(reversed (l)) for l in give_pathes([], end_idx)]

In [32]:
%%time
traintestpath=traintest.apply(lambda x: BFSmulti(x.AdjMatrix, x.atom_index_0, x.atom_index_1), axis=1)
traintestpath.to_pickle('data/traintestpathmulti.pkl')

CPU times: user 34min 45s, sys: 8min 21s, total: 43min 6s
Wall time: 32min 4s
